# Let's use a Data Loader now

In [1]:
from torchvision import datasets
data_path = '../data-unversioned/p1ch7/'
cifar10 = datasets.CIFAR10(data_path, train=True, download=True)
cifar10_val = datasets.CIFAR10(data_path, train=False, download=True)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
import torch
imgs = torch.stack([img_t for img_t, _ in tensor_cifar10], dim=3)
mean = imgs.view(3, -1).mean(dim=1)
std = imgs.view(3, -1).std(dim=1)

transformed_cifar10 = datasets.CIFAR10(
        data_path, train=True, download=False,
        transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean,
                std)
        ]))
transformed_cifar10_test = datasets.CIFAR10(
    data_path, train=False, download=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])
)

In [4]:
label_map = {0: 0, 2: 1}
class_names = ['airplane', 'bird']
cifar2 = [(img, label_map[label]) for img, label in transformed_cifar10
    if label in [0, 2]]
cifar2_val = [(img, label_map[label]) for img, label in transformed_cifar10_test
    if label in [0, 2]]

In [6]:
import torch
import torch.nn as nn

train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64, shuffle=True)
model = nn.Sequential(
    nn.Linear(3072, 512),
    nn.Tanh(),
    nn.Linear(512, 2),
    nn.LogSoftmax(dim=1)
)

Now to the training loop:

In [8]:
import torch.optim as optim

learning_rate = 1e-2
optimizer = optim.SGD(model.parameters(), lr=learning_rate)
loss_fn = nn.NLLLoss()
n_epochs = 100

for epoch in range(n_epochs):
    for imgs, labels in train_loader:
        batch_size = imgs.shape[0]
        outputs = model(imgs.view(batch_size, -1))
        loss = loss_fn(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print("Epoch: %d, Loss: %f" % (epoch, float(loss)))

Epoch: 0, Loss: 0.818551
Epoch: 1, Loss: 0.693236
Epoch: 2, Loss: 0.466720
Epoch: 3, Loss: 0.627785
Epoch: 4, Loss: 0.409150
Epoch: 5, Loss: 0.308183
Epoch: 6, Loss: 0.396919
Epoch: 7, Loss: 0.413389
Epoch: 8, Loss: 0.350740
Epoch: 9, Loss: 0.209187
Epoch: 10, Loss: 0.240583
Epoch: 11, Loss: 0.264094
Epoch: 12, Loss: 0.486026
Epoch: 13, Loss: 0.529498
Epoch: 14, Loss: 0.693913
Epoch: 15, Loss: 0.487888
Epoch: 16, Loss: 0.159273
Epoch: 17, Loss: 0.315060
Epoch: 18, Loss: 0.262060
Epoch: 19, Loss: 0.164396
Epoch: 20, Loss: 0.369528
Epoch: 21, Loss: 0.145587
Epoch: 22, Loss: 0.178740
Epoch: 23, Loss: 0.247337
Epoch: 24, Loss: 0.610965
Epoch: 25, Loss: 0.195835
Epoch: 26, Loss: 0.256968
Epoch: 27, Loss: 0.186783
Epoch: 28, Loss: 0.136518
Epoch: 29, Loss: 0.248617
Epoch: 30, Loss: 0.170237
Epoch: 31, Loss: 0.403260
Epoch: 32, Loss: 0.112144
Epoch: 33, Loss: 0.078118
Epoch: 34, Loss: 0.187679
Epoch: 35, Loss: 0.086263
Epoch: 36, Loss: 0.152157
Epoch: 37, Loss: 0.215807
Epoch: 38, Loss: 0.230

We notice that training in batches is not only faster but also it seems to give us a smaller loss to begin with.

Okay now let's thest the model

In [15]:
val_loader = torch.utils.data.DataLoader(cifar2_val, batch_size=64, shuffle=False)
correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in val_loader:
        batch_size = imgs.shape[0]
        outputs = model(imgs.view(batch_size, -1))
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())
        
    print("Accuracy: ", str(correct / total * 100) + '%')

Accuracy:  81.39999999999999%


81.39% is a great accuracy but I belive we could do better, by improving the model.